<a href="https://colab.research.google.com/github/khanhlinhbk/Semantic-Text-Similarity-With-Bert/blob/master/BERT_BiLSTM_nunfreze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 15.3MB/s 
     |████████████████████████████████| 901kB 56.3MB/s 
     |████████████████████████████████| 3.3MB 59.9MB/s 


In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 165 µs (started: 2021-06-20 09:14:43 +00:00)


In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

time: 3.74 s (started: 2021-06-20 09:14:45 +00:00)


In [ ]:
import pickle
import tempfile
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from tensorflow.keras.layers import Dense

# Hotfix function
def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = {'model_str': model_str}
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

# Run the function
make_keras_picklable()

time: 11.4 ms (started: 2021-06-20 09:14:50 +00:00)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving questions.csv to questions.csv
time: 13min 57s (started: 2021-06-20 09:14:54 +00:00)


In [ ]:
import io
df = pd.read_csv(io.StringIO(uploaded['questions.csv'].decode('utf-8')))
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404346,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404347,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1
404348,404348,789796,789797,What is one coin?,What's this coin?,0
404349,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


time: 1.3 s (started: 2021-06-20 09:28:54 +00:00)


In [ ]:
df.drop(['id', 'qid1', 'qid2'], axis=1, inplace=True)
df.columns = ['question1', 'question2', 'label']

time: 31.1 ms (started: 2021-06-20 09:28:59 +00:00)


In [ ]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

time: 190 ms (started: 2021-06-20 09:29:03 +00:00)


In [ ]:
test

,question1,question2,label
301066,How should I talk to the girl I like?,How do you go up and talk to a girl you like?,1
51450,Who has the best military?,Who has the best military in the world?,1
325180,How does one become a mercenary?,How do I become a mercenary?,1
191549,How did buying and/or selling a home become so...,How do I market or increase visibility of a bu...,0
281839,Why use demand draft?,What is the difference between chalan and dema...,0
...,...,...,...
259178,If universe is expanding without a limit and d...,How is dark/vacuum energy created with the uni...,1
365838,What is the superstition behind a twitching ri...,What is the superstition of the lower left eye...,1
131932,Does Quora form intellectual bubbles?,Who owns the intellectual property of Quora?,0
146867,I just had a very bad break up. My boyfriend l...,What did you do when your ex moved on?,0


time: 22.5 ms (started: 2021-06-20 09:29:05 +00:00)


In [ ]:
y_train = tf.keras.utils.to_categorical(train.label, num_classes=2)
y_validate = tf.keras.utils.to_categorical(validate.label, num_classes=2)
y_test = tf.keras.utils.to_categorical(test.label, num_classes=2)

time: 8.53 ms (started: 2021-06-20 09:29:10 +00:00)


In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 5

time: 1.05 ms (started: 2021-06-20 09:29:11 +00:00)


In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

time: 30.6 ms (started: 2021-06-20 09:29:14 +00:00)


In [ ]:
from tensorflow import keras
from keras.layers import Dense
from transformers import BertConfig;

time: 241 ms (started: 2021-06-20 09:29:17 +00:00)


In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True, output_attentions=True)
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased",config=config)
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = True

    bert_output = bert_model(
    input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    bert_output = tf.concat([bert_output[2][-i] for i in range(4)], axis=-1)
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(bert_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss="binary_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
train_data = BertSemanticDataGenerator(
    train[["question1", "question2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    validate[["question1", "question2"]].values.astype("str"),
    y_validate,
    batch_size=batch_size,
    shuffle=False,
)


time: 7.82 s (started: 2021-06-20 09:30:19 +00:00)


In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
7581/7581 [==============================] - ETA: 0s - loss: 0.3338 - acc: 0.8475

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


7581/7581 [==============================] - 2469s 323ms/step - loss: 0.3338 - acc: 0.8475 - val_loss: 0.2811 - val_acc: 0.8742
Epoch 2/5
7581/7581 [==============================] - 2444s 322ms/step - loss: 0.2309 - acc: 0.9019 - val_loss: 0.2509 - val_acc: 0.8924
Epoch 3/5
7581/7581 [==============================] - 2440s 322ms/step - loss: 0.1686 - acc: 0.9323 - val_loss: 0.2548 - val_acc: 0.8935
Epoch 4/5
7581/7581 [==============================] - 2442s 322ms/step - loss: 0.1201 - acc: 0.9543 - val_loss: 0.2731 - val_acc: 0.8984
Epoch 5/5
7581/7581 [==============================] - 2454s 324ms/step - loss: 0.0866 - acc: 0.9680 - val_loss: 0.3110 - val_acc: 0.9001
time: 3h 24min 8s (started: 2021-06-20 09:30:29 +00:00)


In [ ]:
test_data = BertSemanticDataGenerator(
    test[["question1", "question2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


2527/2527 [==============================] - 275s 109ms/step - loss: 0.3134 - acc: 0.8993


[0.3133814334869385, 0.8992629647254944]

time: 4min 37s (started: 2021-06-20 12:54:44 +00:00)


In [ ]:
labels = [1, 0]

time: 975 µs (started: 2021-06-20 12:59:54 +00:00)


In [ ]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data)[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    if pred == 1:
      label = "Similar"
    else:
      label = "Not Similar"
    print("Prediction: {0}\nProbability: {1}\n\n".format(label, proba))

time: 5.41 ms (started: 2021-06-20 12:59:57 +00:00)


In [ ]:
sentence1 = "How do I prepare for IBPS clerk Mains 2016 exam?"
sentence2 = "How do I prepare for IBPS clerk Mains 2016?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Not Similar
Probability:  1.00%


time: 1.95 s (started: 2021-06-20 14:23:26 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing a football"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Similar
Probability:  1.00%


time: 1.96 s (started: 2021-06-20 13:00:09 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
sentence1 = "Does that dog belong to you?"
sentence2 = "Is that dog yours?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Similar
Probability:  1.00%


time: 1.94 s (started: 2021-06-20 13:00:14 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
sentence1 = "Who's your favorite writer and what book of theirs would you recommend for reading?"
sentence2 = "What is your favorite book of all time and why?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Not Similar
Probability:  0.98%


time: 1.97 s (started: 2021-06-20 13:01:16 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
sentence1 = "What are the strongest majors in terms of job prospects and what are the weakest majors at Ohio University?"
sentence2 = "What are the strongest majors in terms of job prospects and what are the weakest majors at The Ohio State University?"
check_similarity(sentence1, sentence2)

NameError: ignored

In [ ]:
sentence1 = "What is the best/most memorable thing you've ever eaten and why?"
sentence2 = "What is the most delicious dish you've ever eaten and why?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Not Similar
Probability:  1.00%


time: 1.99 s (started: 2021-06-20 13:15:59 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
sentence1 = "I just had a very bad break up. My boyfriend left me without giving any reason. He ended our 7-year strong relationship just because of a caste difference. He is not taking a stand for me. I am shattered. I am dying from the inside. What should I do?"
sentence2 = "What did you do when your ex moved on?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Similar
Probability:  1.00%


time: 1.94 s (started: 2021-06-20 13:28:59 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Sentence pair wrong similarity

In [ ]:
sentence1 = "What powers are denied to the federal government?"
sentence2 = "What are the powers delegated to the federal government?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Not Similar
Probability:  0.86%


time: 1.94 s (started: 2021-06-20 13:53:31 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
sentence1 = "Is Google a good pay master?"
sentence2 = "How much does Google pay a new grad software engineer?"
check_similarity(sentence1, sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prediction: Similar
Probability:  1.00%


time: 1.97 s (started: 2021-06-20 14:14:05 +00:00)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2111: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train1, validate1, test1 = \
              np.split(test.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

time: 32.4 ms (started: 2021-06-20 14:05:06 +00:00)


In [ ]:
train1

,question1,question2,label
139084,What's the best birthday present?,What is the best birthday present someone has ...,0
134759,Exchange programs fro engineering students?,After taking rat poison how long it will take ...,0
51356,When and how will North Korea collapse and imp...,How will North Korea collapse?,1
92525,Is there life on other planets?,What are some signs of life on other planets?,1
164197,How much would it cost to build a pier in NYC?,How much does it cost to build a pier?,0
...,...,...,...
252607,Why there are certain things that make you hap...,Psychology of Everyday Life: What are some lit...,0
4696,Who are the best Internet service providers in...,What's the best residential ISP in San Francisco?,1
143252,"What does ""kanpai"" mean in Japanese?",Japanese (language): What does Bricco (burikko...,0
324866,When and how did you start smoking cigarette?,What is the best way to start smoking cigarettes?,0


time: 24.2 ms (started: 2021-06-20 14:05:09 +00:00)
